## Load Amazon dataset

In [103]:
import pandas as pd
import numpy as np

In [ ]:
products = pd.read_csv('c://users/intel/Desktop/Coursera/Classification/week1/amazon_baby.csv')

In [82]:
type(products)

pandas.core.frame.DataFrame

### Perform text cleaning

In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [8]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [33]:
def remove_punctuation(text):
    import string
    new_text = text
    for i in string.punctuation:
        new_text = new_text.str.replace(i, '')
    return new_text

In [34]:
# test
remove_punctuation(pd.Series('grege! reg, 43'))

0    grege reg 43
dtype: object

In [35]:
products['review_clean'] = remove_punctuation(products['review'])

In [37]:
#check first row
products['review'][1], products['review_clean'][1]

('it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.',
 'it came early and was not disappointed i love planet wise bags and now my wipe holder it keps my osocozy wipes moist and does not leak highly recommend it')

### Extract Sentiments

In [38]:
products = products[products['rating'] != 3]

In [39]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

### Split into training and test sets

In [46]:
test_index = pd.read_json('c://users/intel/Desktop/Coursera/Classification/week1/module-2-assignment-test-idx.json')
train_index = pd.read_json('c://users/intel/Desktop/Coursera/Classification/week1/module-2-assignment-train-idx.json')

In [72]:
products['ind'] = list(range(len(products.index)))

In [88]:
test_data = products.iloc[test_index[0]]
train_data = products.iloc[train_index[0]]

### Build the word count vector for each review

In [89]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

### Train a sentiment classifier with logistic regression

In [93]:
from sklearn import linear_model
model = linear_model.LogisticRegression(fit_intercept=True)
sentiment_model = model.fit(train_matrix, train_data['sentiment'])

In [115]:
np.count_nonzero( np.where(sentiment_model.coef_ >= 0)[1] )

87042

In [117]:
sentiment_model.intercept_

array([ 1.37940741])

## Making predictions with logistic regression

In [118]:
sample_test_data = test_data[10:13]
print(sample_test_data)

                                                 name  \
59                          Our Baby Girl Memory Book   
71  Wall Decor Removable Decal Sticker - Colorful ...   
91  New Style Trailing Cherry Blossom Tree Decal R...   

                                               review  rating  \
59  Absolutely love it and all of the Scripture in...       5   
71  Would not purchase again or recommend. The dec...       2   
91  Was so excited to get this product for my baby...       1   

                                         review_clean  sentiment  ind  
59  Absolutely love it and all of the Scripture in...          1   53  
71  Would not purchase again or recommend The deca...         -1   64  
91  Was so excited to get this product for my baby...         -1   82  


In [124]:
sample_test_data['review'].iloc[0]

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [125]:
sample_test_data['review'].iloc[1]

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [126]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60154242  -3.17071043 -10.42408539]


In [128]:
def predicted_labels(dataset):
    labels= []
    for i in dataset:
        if i > 0:
            labels.append(1)
        else:
            labels.append(-1)
    return labels

In [129]:
predicted_labels(scores)

[1, -1, -1]

In [127]:
#Checkpoint: Make sure your class predictions match with the ones obtained from sentiment_model. 
#The logistic regression classifier in scikit-learn comes with the predict function for this purpose.
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [132]:
import math

In [146]:
def probabilities(dataset):
    probs = []
    for i in dataset:
        probs.append( 1/( 1+math.exp(-i) ) )
    return probs

In [147]:
probabilities(scores)

[0.9963214174803338, 0.04028294108606697, 2.970737868128866e-05]

In [148]:
# Checkpoint: Make sure your probability predictions match the ones obtained from sentiment_model.
sentiment_model.predict_proba(sample_test_matrix)

array([[  3.67858252e-03,   9.96321417e-01],
       [  9.59717059e-01,   4.02829411e-02],
       [  9.99970293e-01,   2.97073787e-05]])

### Find the most positive (and negative) review

In [150]:
test_predictions = sentiment_model.predict(test_matrix)

In [158]:
test_scores = sentiment_model.decision_function(test_matrix)

In [176]:
test_data['probs'] = pd.Series(probabilities(test_scores))

C:\Users\Intel\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [163]:
test_probs.sort()

NameError: name 'desc' is not defined

In [168]:
test_probs[-20:]

[0.9999999999999996,
 0.9999999999999998,
 0.9999999999999998,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]